In [1]:
from typing import Dict

import requests
import pandas as pd
from geojson_pydantic import Feature, Polygon

In [5]:
BASE_URL = "https://space2stats.ds.io"
FIELDS_ENDPOINT = f"{BASE_URL}/fields"
SUMMARY_ENDPOINT = f"{BASE_URL}/summary"

In [6]:
AOIModel = Feature[Polygon, Dict]

# ~kenya
aoi = {
    "type": "Feature",
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [33.78593974945852, 5.115816884114494],
                [33.78593974945852, -4.725410543134203],
                [41.94362577283266, -4.725410543134203],
                [41.94362577283266, 5.115816884114494],
                [33.78593974945852, 5.115816884114494]
            ]
        ]
    },
    "properties": {
        "name": "Updated AOI"
    }
}
          
  

feat = AOIModel(**aoi)

In [9]:
# Define the Request Payload
request_payload = {
    "aoi": aoi,
    "spatial_join_method": "centroid",
    "fields": ["sum_pop_2020"], 
    "geometry": "point"
}

# Get Summary Data
response = requests.post(SUMMARY_ENDPOINT, json=request_payload)
if response.status_code != 200:
    raise Exception(f"Failed to get summary: {response.text}")

summary_data = response.json()
df = pd.DataFrame(summary_data)
df.head()

,hex_id,geometry,sum_pop_2020
0,866a4a017ffffff,"{'type': 'Point', 'coordinates': [37.844379308...",676.912804
1,866a4a01fffffff,"{'type': 'Point', 'coordinates': [34.732706798...",347.182722
2,866a4a04fffffff,"{'type': 'Point', 'coordinates': [38.771304043...",285.943490
3,866a4a057ffffff,"{'type': 'Point', 'coordinates': [41.630359262...",279.463912
4,866a4a05fffffff,"{'type': 'Point', 'coordinates': [39.038040116...",235.258474


### Aggregate Results

In some cases, the analyst and researchers may be interested only in the summary statistics for the entire region of interest. In this section, we demonstrate how users can easily aggregate the values of the hexagons for their region of interest.

	We strongly recommend being mindful of data that may be lost in this two-step aggregation process (raw data to H3 hexagons to aggregate information). Since each hexagon is given equal weight during aggregation, this can introduce inaccuracies for certain variables. For example, aggregating population data may be reasonable because each hexagon contributes equally to the total population. However, for other variables, such as ratios (e.g., the ratio of men to women), this approach may produce skewed results. The aggregated value may differ significantly compared to what you would get by aggregating the raw data directly. In such cases, it’s important to assess whether the variable should be aggregated at the hexagon level or if a different approach would yield more accurate results.



In [10]:
mean_sum_pop_2020 = df['sum_pop_2020'].mean()
total_pop_2020 = df['sum_pop_2020'].sum()
print(f"Mean population per hexagon: {mean_sum_pop_2020}")
print(f"Total population for region: {total_pop_2020}")

Mean population per hexagon: 2865.592428932978
Total population for region: 73949478.22104442
